## Here I want to do several things
The first routine should take a given a set of points in 3D and compute the list of all possible tetrahedrons.

The second routine should take a second set of 3D points and the list of all tetrahedrons computed before and computes their barycentric coordinates for the tetrahedrons containing them.

I did refresh my memory by reading this http://www.iue.tuwien.ac.at/phd/nentchev/node31.html

In [1]:
from scipy.spatial import ConvexHull
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import scipy.spatial

### List of all possible tethrahedrons
The first routine should take a given a set of points in 3D and compute the list of all possible tetrahedrons.

In [6]:
#generate some random test points
N_A = 10 # like 10 points...
set_3D_points_A = np.random.rand(N_A,3) # all values between [0,1]
print set_3D_points_A[0:3,:], np.shape(set_3D_points_A) # only show the first 3 points

[[ 0.84613296  0.0728177   0.68630627]
 [ 0.43849045  0.02387736  0.12405254]
 [ 0.13263709  0.14567459  0.05800825]] (10, 3)


We start with a definition, a tetrahedron is made of four points/vertex in 3D spaces, it has four triangle faces.

So if I have *N* points, then all possible combinations of *4* of them give me the list of possible tetrahedrons.

The function below tries to do so using all possible combinations.

In [7]:
def fun_list_all_tetrahedron_from_set_of_points(set_3D_points):
    """
    The function does what its name says.
    It takes a list of a least four points in 3D space and return
    all possible combinations of four points defining a tetrahedron.
    """
    # get the list of indexs for each tetrahedron
    N = np.shape(set_3D_points)[0]
    vec_N = np.arange(0,N)
    
    # get the coordinates for each list of tetrahedra 
    nb_combibation = itertools.combinations(vec_N, 4)
    list_combination = np.array(list(nb_combibation))
    #tetrahedron_coordinates = set_3D_points[nb_combibation,:]
    return list_combination


**nb_combibation** gives the index numbers for each group of four points defining a tetrahedron.

This approach didn't test if for a set of four points the points were in the same plan and didn't tell if the tetrahedrons were exclusive or not in the sens no points were located within a tetrahedron. You may want to know how many unique tetrahedron there are in your set of points.

Some test data below to try the function.

In [8]:
list_index_tetra = fun_list_all_tetrahedron_from_set_of_points(set_3D_points_A)
print len(list_index_tetra),list_index_tetra[0:5,:], np.shape(list_index_tetra)
print "For a set of %1.0f points we have %1.0f tetrahedronds" % (N_A,np.shape(list_index_tetra)[0])

210 [[0 1 2 3]
 [0 1 2 4]
 [0 1 2 5]
 [0 1 2 6]
 [0 1 2 7]] (210, 4)
For a set of 10 points we have 210 tetrahedronds


## Compute the barycentric coordinates for the tetrahedrons
The second routine should take a second set of 3D points and the list of all tetrahedrons computed before and computes their barycentric coordinates for the tetrahedrons containg them.

In [19]:
N_B = 9
set_3D_points_B = np.random.rand(N_B,3) # all values between [0,1]
set_3D_points_B = np.vstack([set_3D_points_B, [0.5, 0.5, 0.5]])
print set_3D_points_B[-3:,:]

[[ 0.05637844  0.61356779  0.59889278]
 [ 0.6459152   0.77118909  0.96132632]
 [ 0.5         0.5         0.5       ]]


I basically use the scipy spatial function here... What I do is the Delaunay triangulation to *clean* the number of good tetrahedrons. It also makes it easier to use the function from scipy.

In [20]:
# Delaunay triangulation
tri = scipy.spatial.Delaunay(set_3D_points_A)
print "I have %1.0f tetrahedra after the triangulation. " % np.shape(tri.simplices)[0]

I have 19 tetrahedra after the triangulation. 


In [30]:
# find which point are in which tetrahedron
tetrahedra = tri.find_simplex(set_3D_points_B)
print tetrahedra

# find the 
X = tri.transform[tetrahedra,:3]
Y = set_3D_points_B - tri.transform[tetrahedra,3]
b = np.einsum('ijk,ik->ij', X, Y)
bcoords = np.c_[b, 1 - b.sum(axis=1)]


[18 17 -1 -1 -1 -1 -1 -1 -1  1]


In [35]:
# display the barycentric coordinate for each point in its corresponding tetrahedra
for tt in np.arange(len(tetrahedra)):
    if tetrahedra[tt] > 0: # then the point is in a tetrahedron
        print "indexes of tetradron"
        print tri.simplices[tt,:], set_3D_points_B[tt,:]
        print set_3D_points_A[tri.simplices[tt,:],:]

indexes of tetradron
[6 8 2 5] [ 0.41773754  0.32913475  0.8339528 ]
[[ 0.62809736  0.11262766  0.52908443]
 [ 0.36981401  0.84670651  0.57888926]
 [ 0.13263709  0.14567459  0.05800825]
 [ 0.12761632  0.56675243  0.995321  ]]
indexes of tetradron
[6 8 3 2] [ 0.35382837  0.72984755  0.8041507 ]
[[ 0.62809736  0.11262766  0.52908443]
 [ 0.36981401  0.84670651  0.57888926]
 [ 0.55193841  0.69269942  0.35854947]
 [ 0.13263709  0.14567459  0.05800825]]
indexes of tetradron
[4 6 3 0] [ 0.5  0.5  0.5]
[[ 0.75897004  0.47295987  0.76712544]
 [ 0.62809736  0.11262766  0.52908443]
 [ 0.55193841  0.69269942  0.35854947]
 [ 0.84613296  0.0728177   0.68630627]]
